# Visualize Trends in Return Values for a 1-in-10-year Event
---
**Project**: Masters Project <br>
**Author**: Nabig Chaudhry

In [2]:
# import necessary packages

import requests 
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import os as os
from scipy import stats
import statsmodels.api as sm

import lmoments3 as lm
from lmoments3 import distr as ldistr

import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
from bokeh.io import export_svgs

In [3]:
# import functions specifically for masters_project

%load_ext autoreload
%autoreload 2

import masters_project_functions as mp

## Step 1: Pull Final Return Value Data For Analysis

In [4]:
os.listdir('./data/intermediate_processed')

['cnrm_ssp370_2060.nc',
 'cesm2_ssp370_2040.nc',
 'cesm2_ssp370_2060.nc',
 'cnrm_ssp370_2040.nc',
 'cesm2_ssp370_2020.nc',
 'cnrm_ssp370_2020.nc',
 'cnrm_hist_1980.nc',
 'cnrm_ssp370_2080.nc',
 'cesm2_hist_1980.nc',
 'cesm2_ssp370_2080.nc',
 '.ipynb_checkpoints']

In [5]:
import_path = './data/final_for_analysis/return_value'

In [6]:
cesm2_hist_1980_rv = xr.open_dataset(f'{import_path}/cesm2_hist_1980_rv.nc')
cesm2_ssp370_2020_rv = xr.open_dataset(f'{import_path}/cesm2_ssp370_2020_rv.nc')
cesm2_ssp370_2040_rv = xr.open_dataset(f'{import_path}/cesm2_ssp370_2040_rv.nc')
cesm2_ssp370_2060_rv = xr.open_dataset(f'{import_path}/cesm2_ssp370_2060_rv.nc')
cesm2_ssp370_2080_rv = xr.open_dataset(f'{import_path}/cesm2_ssp370_2080_rv.nc')

cnrm_hist_1980_rv = xr.open_dataset(f'{import_path}/cnrm_hist_1980_rv.nc')
cnrm_ssp370_2020_rv = xr.open_dataset(f'{import_path}/cnrm_ssp370_2020_rv.nc')
cnrm_ssp370_2040_rv = xr.open_dataset(f'{import_path}/cnrm_ssp370_2040_rv.nc')
cnrm_ssp370_2060_rv = xr.open_dataset(f'{import_path}/cnrm_ssp370_2060_rv.nc')
cnrm_ssp370_2080_rv = xr.open_dataset(f'{import_path}/cnrm_ssp370_2080_rv.nc')

## Step 2: Pull Return Value Arrays for CESM2 Global Climate Model

In [7]:
cesm2_hist_1980_rv_array = mp.get_flat_array(cesm2_hist_1980_rv, 
                                             data_variable='return_value')

In [8]:
cesm2_ssp370_2020_rv_array = mp.get_flat_array(cesm2_ssp370_2020_rv, 
                                             data_variable='return_value')

In [9]:
cesm2_ssp370_2040_rv_array = mp.get_flat_array(cesm2_ssp370_2040_rv, 
                                             data_variable='return_value')

In [10]:
cesm2_ssp370_2060_rv_array = mp.get_flat_array(cesm2_ssp370_2060_rv, 
                                             data_variable='return_value')

In [11]:
cesm2_ssp370_2080_rv_array = mp.get_flat_array(cesm2_ssp370_2080_rv, 
                                             data_variable='return_value')

In [64]:
# create dataframe from all return value arrays

cesm2_rv_df = pd.DataFrame({'cesm2_hist_1980': cesm2_hist_1980_rv_array, 
                            'cesm2_ssp370_2020': cesm2_ssp370_2020_rv_array,
                            'cesm2_ssp370_2040': cesm2_ssp370_2040_rv_array,
                            'cesm2_ssp370_2060': cesm2_ssp370_2060_rv_array,
                            'cesm2_ssp370_2080': cesm2_ssp370_2020_rv_array})

## Step 3: Pull Return Value Arrays for CNRM-ESM2-1 Global Climate Model

In [25]:
cnrm_hist_1980_rv_array = mp.get_flat_array(cnrm_hist_1980_rv, 
                                             data_variable='return_value')

In [26]:
cnrm_ssp370_2020_rv_array = mp.get_flat_array(cnrm_ssp370_2020_rv, 
                                             data_variable='return_value')

In [27]:
cnrm_ssp370_2040_rv_array = mp.get_flat_array(cnrm_ssp370_2040_rv, 
                                             data_variable='return_value')

In [28]:
cnrm_ssp370_2060_rv_array = mp.get_flat_array(cnrm_ssp370_2060_rv, 
                                             data_variable='return_value')

In [29]:
cnrm_ssp370_2080_rv_array = mp.get_flat_array(cnrm_ssp370_2080_rv, 
                                             data_variable='return_value')

In [65]:
# create dataframe from all return value arrays

cnrm_rv_df = pd.DataFrame({'cnrm_hist_1980': cnrm_hist_1980_rv_array, 
                            'cnrm_ssp370_2020': cnrm_ssp370_2020_rv_array,
                            'cnrm_ssp370_2040': cnrm_ssp370_2040_rv_array,
                            'cnrm_ssp370_2060': cnrm_ssp370_2060_rv_array,
                            'cnrm_ssp370_2080': cnrm_ssp370_2080_rv_array})

## Step 4: Set Graphing Parameters

In [117]:
box_fill_color = 'orange'
box_hover_color = 'blue'

width=650
height=400

columns = ['Historical 1980-99',
          'SSP3-7.0 2020-39',
          'SSP3-7.0 2040-59',
          'SSP3-7.0 2060-79',
          'SSP3-7.0 2080-99']

group_label='Climate Models'
value_label='Return Values (Degrees C)'

## Step 5: Visualize Trends and Return Value Data Through Box Plots

In [122]:
df = cesm2_rv_df.rename(columns={'cesm2_hist_1980':'Historical 1980-99', 
                                'cesm2_ssp370_2020':'SSP3-7.0 2020-39', 
                                'cesm2_ssp370_2040':'SSP3-7.0 2040-59',
                                'cesm2_ssp370_2060':'SSP3-7.0 2060-79',
                                'cesm2_ssp370_2080':'SSP3-7.0 2080-99'})

title = 'CESM2 Return Values for 1-in-10-year Event'

cesm2_box_plot = df.hvplot.box(y=columns,
                               width=width,
                               height=height,
                               title=title,
                               box_fill_color=box_fill_color,
                               box_hover_color=box_hover_color,
                               group_label=group_label,
                               value_label=value_label)
cesm2_box_plot

:BoxWhisker   [Climate Models]   (Return Values (Degrees C))

In [123]:
df = cnrm_rv_df.rename(columns={'cnrm_hist_1980':'Historical 1980-99', 
                                'cnrm_ssp370_2020':'SSP3-7.0 2020-39', 
                                'cnrm_ssp370_2040':'SSP3-7.0 2040-59',
                                'cnrm_ssp370_2060':'SSP3-7.0 2060-79',
                                'cnrm_ssp370_2080':'SSP3-7.0 2080-99'})

title = 'CNRM-ESM2-1 Return Values for 1-in-10-year Event'

cnrm_box_plot = df.hvplot.box(y=columns,
                              width=width,
                              height=height,
                              title=title,
                              box_fill_color=box_fill_color,
                              box_hover_color=box_hover_color,
                              group_label=group_label,
                              value_label=value_label)
cnrm_box_plot

:BoxWhisker   [Climate Models]   (Return Values (Degrees C))

## Step 4: Export Percent Change Visualizations

In [124]:
export_stat_image_path = './images/static/return_value'

In [125]:
# define function to export visualizations as static svg files

def export_svg(obj, filename):
    plot_state = hv.renderer('bokeh').get_plot(obj).state
    plot_state.output_backend = 'svg'
    export_svgs(plot_state, filename=filename)

In [126]:
# export visualizations as static svg images

export_svg(cesm2_box_plot, f'{export_stat_image_path}/cesm2_box_plot.svg')
export_svg(cnrm_box_plot, f'{export_stat_image_path}/cnrm_box_plot.svg')